<a href="https://colab.research.google.com/github/VctrEman/tesourodireto/blob/main/TesouroDiretoReport.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pretty_html_table

import pandas as pd
import requests
from datetime import timedelta
import smtplib
from pretty_html_table import build_table
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText        
from email.mime.base import MIMEBase        #biblioteca nao utilizada, avaliar
from email import encoders                  #biblioteca nao utilizada, avaliar
import email.message                        #biblioteca nao utilizada, avaliar

In [2]:
def GetOauth():             
    #Requests authorization response to to enter on Google
    payload =  """{\r\n\"client_secret\":\"H-WwaYIMP4MFmP-RJE2-5X_J\",
            \r\n\"grant_type\": \"refresh_token\",
            \r\n\"refresh_token\":\"1//04yt5o-0GTEZMCgYIARAAGAQSNwF-L9IrQu5mTb2H_hf4zKXb1N2xcrhTm3XGe9_6b1hw9AKks9JvGP1Xpc3WfNe1Dor1WZ007aw\",
            \r\n\"client_id\":\"932267562254-3t83u399s52cnj204kilqubvb5q65cv4.apps.googleusercontent.com\"\r\n}
            \r\n"""
    headers = {'Content-Type': 'text/plain'}
    
    response = requests.request("POST","https://accounts.google.com/o/oauth2/token", headers= headers, data=payload).json()
    return response['access_token']

def get_google_sheet_df(headers: dict, google_sheet_id: str, sheet_name: str, _range: str):
    #Requests data from Google Sheets API
    url = f'https://sheets.googleapis.com/v4/spreadsheets/{google_sheet_id}/values/{sheet_name}!{_range}'
    r = requests.get(url, headers=headers)
    values = r.json()['values']
    df = pd.DataFrame(values[1:])
    df.columns = values[0]
    return df

def main_request():
    headers = {'Authorization': "Bearer "+GetOauth(),'Content-Type': 'application/vnd.api+json'}
    google_sheet_id = '1yXhAFxsaSb_X1WTPgUodswv8OZ0cHnXAqjTVFUS3lWs'
    sheet_name = 'Sheet1'
    sample_range = 'A:M'

    dfgsheets = get_google_sheet_df(headers, google_sheet_id, sheet_name, sample_range)  #request do gsheets
    return dfgsheets

def tabela_acimadameta(): #JESSE
    df = main_request()
    df['Refresh Date'] = df['Refresh Date'].astype('datetime64[ns]')
    df['Taxa de Juros'] = df['Taxa de Juros'].astype(float)

    tx_juros = df.pivot_table(values='Taxa de Juros', index='Refresh Date', columns='codigo', aggfunc='last')
    dataref = (tx_juros.iloc[[-1]].index - timedelta(hours=3)).strftime("%d/%m/%Y %H:%M:%S")[0] 
    w = 14
    titulos_acima_meta = []
    taxas_acima_meta = []
    vlmeta = []
    for column in tx_juros:
        sigma_tx_juros = (tx_juros[ column ].rolling(window= w ).mean() + tx_juros[ column ].rolling(window= w ).std()*(1.5))[-1]
        today_tx_juros = tx_juros[ column ].iloc[[-1]].values  
        if sigma_tx_juros <= today_tx_juros: 
            titulos_acima_meta += [column]
            vlmeta += [sigma_tx_juros]
            taxas_acima_meta   += [tx_juros[column][-1]]

    list(zip(titulos_acima_meta, vlmeta, taxas_acima_meta))
    new_df = pd.DataFrame(data={'codigo': titulos_acima_meta, 'valor atual': taxas_acima_meta, 'valor critério': vlmeta})
    new_df['nome'] = new_df['codigo'].apply(lambda x: df[df['codigo'] == x]['Titulo'].values[0])
    return new_df, dataref

def send_email(dataframe, now): #FERNANDO
    
    recipients = ["rotcivemanuelms@gmail.com", "rotcivemanuelms@alu.ufc.br", "jessenl@alu.ufc.br", "fernandofares@alu.ufc.br" ] # ***fernandin, a parte de enviar para vários usuarios
    host="smtp.gmail.com"
    port="587"
    login='fernandofares@alu.ufc.br'
    senha='didyppkcigixmgxb'
    server=smtplib.SMTP(host,port)
    server.ehlo()
    server.starttls()
    server.login(login,senha)
    corpo = f"""
        <p>Bom dia!!! Hoje é um ótimo dia para comprar títulos da tabela abaixo</p> Atualizado em: {now} </p> Aproveite essa oportunidade em: https://www.tesourodireto.com.br/ </p>
            """
    email_msg=MIMEMultipart()
    email_msg['From']=login
    email_msg['To']=     ','.join(recipients)
    email_msg['Subject']='NOTIFICAÇÃO DE COMPRA DE TÍTULO'
    email_msg.attach(MIMEText(corpo,'html'))
    output=build_table( dataframe ,"blue_dark",font_size='medium',font_family='Open Sans, sans-serif',text_align='left',width='auto',index=False,even_color='black',even_bg_color='white')
    email_msg.attach(MIMEText(output,'html'))
    server.sendmail(email_msg['From'], email_msg['To'], email_msg.as_string())
    server.quit()

In [4]:
#from email_libs import *

dataframe, now = tabela_acimadameta()
if dataframe.shape[0] > 0:
    send_email(dataframe, now)
    print('email sent succesfully')
else:
    print('no')

email sent succesfully


In [ ]:
#vectorized version 1.5s
def create_table():
    df = main_request()
    df['Refresh Date'] = df['Refresh Date'].astype('datetime64[ns]')
    df['Taxa de Juros'] = df['Taxa de Juros'].astype(float)
    df['Data Vencimento'] = df['Data Vencimento'].astype('datetime64[ns]').dt.strftime("%d/%m/%Y")

    tx_juros = df.pivot_table(values='Taxa de Juros', index='Refresh Date', columns='codigo', aggfunc='last')
    dataref = (df['Refresh Date'].iloc[[-1]]).item()
    now = dataref - timedelta(hours=3)

    df_new = df[['Titulo', 'Valor minimo', 'Taxa de Juros','Data Vencimento']].loc[ (df['Refresh Date'] == dataref ) & \
            (df.codigo.isin(\
            tx_juros[ tx_juros.iloc[[-1]].where(sigma_tx_juros <  today_tx_juros ).dropna(axis = 1).columns].iloc[[-1]].columns))].head()
    return df_new, now
create_table()

(                     Titulo Valor minimo  Taxa de Juros Data Vencimento
 507  Tesouro Prefixado 2024        37.41          11.49      01/07/2024
 508  Tesouro Prefixado 2026        31.49          11.67      01/01/2026,
 Timestamp('2021-10-23 09:45:17'))